In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
/kaggle/input/contract-notes/2022-03-08-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-09-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-07-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-17-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-16-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-15-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-11-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-10-contract-notes_DM3955.pdf
/kaggle/input/contract-notes/2022-03-14-contract-notes_DM3955.pdf


In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import re
from io import StringIO

#Libraries for feature extraction and topic modeling
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### Starting the PDF table Extraction

In [3]:
!pip3 install camelot-py[cv] tabula-py

     |████████████████████████████████| 40 kB 14 kB/s             
     |████████████████████████████████| 12.0 MB 1.8 MB/s            
     |████████████████████████████████| 242 kB 65.9 MB/s            
     |████████████████████████████████| 77 kB 4.1 MB/s             
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 11.7 MB 64.2 MB/s            
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=f1e3e0355851bae92b5fde67bc541070ddbac36b12be62e16c284f56c4c276c3
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


In [4]:
import camelot 
import tabula

In [5]:
!pip install pikepdf

     |████████████████████████████████| 2.3 MB 287 kB/s            


In [6]:
import pikepdf
import pandas as pd
import numpy as np
#pdf = pikepdf.open(file,password='ABDPM5030G')
#pdf.save('book_without_pass.pdf')
import warnings
warnings.filterwarnings('ignore')

In [7]:
def clean_df(my_table):
    my_table = my_table[2:]
    my_table.drop([0,1,3,8,12],axis=1,inplace=True)
    return my_table

In [8]:
def select_tables(ca_table):
    calc_tab = pd.DataFrame()
    for table in ca_table:
        if table.df.shape[1] == 13:
            calc_tab = pd.concat([calc_tab,table.df])
    calc_tab = clean_df(calc_tab)
    return calc_tab

In [9]:
def correct_txn(dataframe):
    for index,trans in enumerate(dataframe.transaction):
        if '\n' in trans:
            txn_len = len(trans.split('\n'))
            #print(index)
            for i in range(txn_len):
                dataframe.loc[index+i,'transaction'] = trans.split('\n')[i]
    return dataframe

In [10]:
def correct_qty(dataframe):
    for index,qty in enumerate(dataframe.qty):
        if '\n' in qty:
            txn_len = len(qty.split('\n'))
            #print(index)
            for i in range(txn_len):
                dataframe.loc[index+i,'qty'] = qty.split('\n')[i]
    return dataframe

In [11]:
#The below function takes the raw file and outputs the cleaned dataframe
def get_data(file):
    pdf = pikepdf.open(file,password='ABTHESOAB')
    pdf.save('book_without_pass.pdf')
    ca_tables = camelot.read_pdf('./book_without_pass.pdf',pages='all')
    my_data = select_tables(ca_tables)
    my_data.columns = ['order_no','script','transaction',
                   'qty','price','price_d','price_close','PnL']
    my_data = my_data[my_data.order_no != '']
    my_data = my_data.reset_index()
    my_data.drop('index',axis=1,inplace=True)
    my_data = correct_qty(my_data)
    my_data = correct_txn(my_data)
    my_data.qty = my_data.qty.astype('int')
    my_data.price = my_data.price.astype('float')
    return my_data # Try to take the raw data as much as possible, to avoid losing control

In [12]:
my_table = get_data('../input/contract-notes/2022-03-08-contract-notes_DM3955.pdf')
my_table

2022-03-18T15:08:05 - INFO - Processing page-1
2022-03-18T15:08:09 - INFO - Processing page-2
2022-03-18T15:08:15 - INFO - Processing page-3
2022-03-18T15:08:19 - INFO - Processing page-4
2022-03-18T15:08:21 - INFO - Processing page-5


,order_no,script,transaction,qty,price,price_d,price_close,PnL
0,0,SILVERM22APRFUT,B,5,70080.0,70080.0000,71442.0000,(1585.00)
1,0,SILVERM22JUNFUT,S,5,70809.0,70809.0000,72218.0000,(5210.00)
2,0,SILVERMIC22APRFUT,B,4,70069.0,70069.0000,71418.0000,(443.00)
3,0,SILVERMIC22JUNFUT,S,4,70758.0,70758.0000,72211.0000,2672.00
4,210000850,SILVERM22APRFUT,S,5,69986.0,69986.0000,71442.0000,349930.00
...,...,...,...,...,...,...,...,...
110,210090301,SILVERMIC22JUNFUT,S,1,71959.0,71959.0000,72211.0000,71959.00
111,210092158,SILVERMIC22JUNFUT,B,1,72194.0,72194.0000,72211.0000,(72194.00)
112,210128021,SILVERMIC22JUNFUT,S,1,72251.0,72251.0000,72211.0000,72251.00
113,210165776,SILVERMIC22JUNFUT,S,1,72323.0,72323.0000,72211.0000,72323.00


In [13]:
#important is check the unique transaction types. Funny things can happen
my_table.transaction.unique()

array(['B', 'S'], dtype=object)

In [14]:
def todays_forward(carry_forward, day_df):
    trade_count = day_df.groupby(['script','transaction'])['qty'].sum().reset_index()
    #Cancelling out the trades of the day
    traded_scripts = trade_count.script.unique()
    transaction = []
    txn_qty = []
    for script in traded_scripts:
        temp = trade_count[trade_count.script == script]
        #which is more?
        if (temp.qty.values[0] > temp.qty.values[-1]):
            x = temp.qty.values[0] - temp.qty.values[-1]
            transaction.append('B')
            txn_qty.append(x)
        else:    
            x = temp.qty.values[-1] - temp.qty.values[0]
            transaction.append('S')
            txn_qty.append(x)
    #Creating the pending trades of the day
    traded_df = pd.DataFrame({'script' : traded_scripts,
                         'txn_type' : transaction,
                         'txn_qty' : txn_qty
                         })
    final_df = traded_df.merge(right=carry_f,left_on='script',right_on='script',how='left')
    final_df['remaining_qty'] = abs(final_df.txn_qty - final_df.qty)
    final_df.drop(['order_no','price','price_d','price_close','PnL'],axis=1,inplace=True)
    return final_df

In [15]:
#decide the script
def locate_script(day_df,script,qty,transaction):
    locating = day_df[(day_df.script == script)&(day_df.transaction == transaction)]
    return locating.iloc[-qty:,:]
            #day_dy[-3:,:] will only work

In [16]:
#Seperating the carry forwarded scripts
carry_f = my_table[my_table.order_no == '0']
carry_f

,order_no,script,transaction,qty,price,price_d,price_close,PnL
0,0,SILVERM22APRFUT,B,5,70080.0,70080.0000,71442.0000,(1585.00)
1,0,SILVERM22JUNFUT,S,5,70809.0,70809.0000,72218.0000,(5210.00)
2,0,SILVERMIC22APRFUT,B,4,70069.0,70069.0000,71418.0000,(443.00)
3,0,SILVERMIC22JUNFUT,S,4,70758.0,70758.0000,72211.0000,2672.00


In [17]:
day_df = my_table[my_table.order_no != '0']
day_df

,order_no,script,transaction,qty,price,price_d,price_close,PnL
4,210000850,SILVERM22APRFUT,S,5,69986.0,69986.0000,71442.0000,349930.00
5,210019577,SILVERM22APRFUT,S,5,70540.0,70540.0000,71442.0000,352700.00
6,210034390,SILVERM22APRFUT,S,5,71129.0,71129.0000,71442.0000,355645.00
7,210086664,SILVERM22APRFUT,B,5,71168.0,71168.0000,71442.0000,(355840.00)
8,210170387,SILVERM22APRFUT,B,5,71720.0,71720.0000,71442.0000,(358600.00)
...,...,...,...,...,...,...,...,...
110,210090301,SILVERMIC22JUNFUT,S,1,71959.0,71959.0000,72211.0000,71959.00
111,210092158,SILVERMIC22JUNFUT,B,1,72194.0,72194.0000,72211.0000,(72194.00)
112,210128021,SILVERMIC22JUNFUT,S,1,72251.0,72251.0000,72211.0000,72251.00
113,210165776,SILVERMIC22JUNFUT,S,1,72323.0,72323.0000,72211.0000,72323.00


In [18]:
today_forward = todays_forward(carry_forward=carry_f,day_df=day_df)
today_forward

,script,txn_type,txn_qty,transaction,qty,remaining_qty
0,SILVERM22APRFUT,S,5,B,5,0
1,SILVERM22JUNFUT,B,5,S,5,0
2,SILVERMIC22APRFUT,S,1,B,4,3
3,SILVERMIC22JUNFUT,B,1,S,4,3


### Now we know which scripts are finally pending at the day's end

Locate the final trades in that script

In [19]:
script_dataframe = pd.DataFrame()
for script in today_forward.loc[today_forward.remaining_qty != 0,'script']:
    qty_tofun = today_forward.loc[today_forward.script == script,'remaining_qty'].values[0]
    #print(qty_tofun)
    txn_tofun = today_forward.loc[today_forward.script == script,'transaction'].values[0]
    #print(txn_tofun)
    temp = locate_script(day_df=day_df,script=script,qty=qty_tofun,transaction=txn_tofun)
    script_dataframe = script_dataframe.append([temp])

In [20]:
script_dataframe.price = script_dataframe.price.astype('float')
script_dataframe.price_close = script_dataframe.price_close.astype('float') 
script_dataframe['PnL'] = script_dataframe.price - script_dataframe.price_close
script_dataframe

,order_no,script,transaction,qty,price,price_d,price_close,PnL
78,210206632,SILVERMIC22APRFUT,B,1,72372.0,72372.0000,71418.0,954.0
80,210232989,SILVERMIC22APRFUT,B,1,72525.0,72525.0000,71418.0,1107.0
81,210238100,SILVERMIC22APRFUT,B,1,72599.0,72599.0000,71418.0,1181.0
110,210090301,SILVERMIC22JUNFUT,S,1,71959.0,71959.0000,72211.0,-252.0
112,210128021,SILVERMIC22JUNFUT,S,1,72251.0,72251.0000,72211.0,40.0
113,210165776,SILVERMIC22JUNFUT,S,1,72323.0,72323.0000,72211.0,112.0


In [21]:
def closing_scripts(file):
    my_table = get_data(file)
    
    my_table = correct_txn(my_table) # correct the transaction errors
    print('corrected transactions')
    carry_f = my_table[my_table.order_no == '0'] # get carry forwarded tables
    
    day_df = my_table[my_table.order_no != '0'] # get day's trade
    
    today_forward = todays_forward(carry_forward=carry_f,day_df=day_df) # get day's forwarded trade table
    print('got today forwarded table')
    #Iterating over the trade table to locate the final trades of the day
    script_dataframe = pd.DataFrame()
    for script in today_forward.loc[today_forward.remaining_qty != 0,'script']:
        qty_tofun = today_forward.loc[today_forward.script == script,'remaining_qty'].values[0]
        #print(qty_tofun)
        txn_tofun = today_forward.loc[today_forward.script == script,'transaction'].values[0]
        #print(txn_tofun)
        temp = locate_script(day_df=day_df,script=script,qty=qty_tofun,transaction=txn_tofun)
        script_dataframe = script_dataframe.append([temp])
    print('Got the script data frame for final processing')
    #final leg of the calculation to get the list of trades getting forwarded
    script_dataframe.price = script_dataframe.price.astype('float')
    script_dataframe.price_close = script_dataframe.price_close.astype('float') 
    script_dataframe['PnL'] = script_dataframe.price - script_dataframe.price_close
    print('returning the solution')
    return script_dataframe

In [22]:
contact11 = get_data('../input/contract-notes/2022-03-11-contract-notes_DM3955.pdf')

2022-03-18T15:08:29 - INFO - Processing page-1
2022-03-18T15:08:33 - INFO - Processing page-2
2022-03-18T15:08:38 - INFO - Processing page-3
2022-03-18T15:08:40 - INFO - Processing page-4


In [23]:
contact17 = get_data('../input/contract-notes/2022-03-17-contract-notes_DM3955.pdf')

2022-03-18T15:08:45 - INFO - Processing page-1
2022-03-18T15:08:49 - INFO - Processing page-2
2022-03-18T15:08:52 - INFO - Processing page-3


In [24]:
contact11.script.unique()

array(['GOLDGUINEA22APRFUT', 'SILVERMIC22APRFUT'], dtype=object)

In [25]:
contact17.script.unique()

array(['SILVERMIC22APRFUT', 'SILVERMIC22JUNFUT'], dtype=object)

In [26]:
temp = contact11[contact11.script == 'SILVERMIC22APRFUT']
buy = temp[temp.transaction == 'B'].reset_index()
buy.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
buy.order_no = buy.order_no.astype('int') 
sell = temp[temp.transaction == 'S'].reset_index()
sell.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
sell.order_no = sell.order_no.astype('int') 

In [27]:
temp = contact17[contact17.script == 'SILVERMIC22APRFUT']
buy = temp[temp.transaction == 'B'].reset_index()
buy.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
buy.order_no = buy.order_no.astype('int') 
sell = temp[temp.transaction == 'S'].reset_index()
sell.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
sell.order_no = sell.order_no.astype('int') 

In [28]:
!pip install xlwt

     |████████████████████████████████| 99 kB 379 kB/s            


In [29]:
import xlwt
sell.to_excel('sell_silver_Apr_raw_17.xls')
buy.to_excel('buy_silver_Apr_raw_17.xls')

Using df.iloc[1] = ['India', 'Shivam', 'Pandey'] can let you add the values to the data frames.

Next question is how to insert rows into the dataframes.. 

df.loc[df.shape[0]] = [None, None, None] this is the interesting twist.. found at 

https://www.stackvidhya.com/add-row-to-dataframe/

In [30]:
#There is a easier and correct method. In this case, the source data frame gets overwritten
def make_single(dataf,qty,index):
    values = dataf.loc[index]
    values[3] = 1 #Here i am changing the qty to 1
    values=pd.DataFrame([values.values],columns=values.index)
    df = pd.DataFrame()
    for q in range(qty):
        df = df.append(values)
    return df

In [31]:
list(buy.loc[2].values)

[210015963, 'SILVERMIC22APRFUT', 'B', 1, 68557.0]

In [32]:
buy.loc[2].values

array([210015963, 'SILVERMIC22APRFUT', 'B', 1, 68557.0], dtype=object)

In [33]:
def make_single_1(dataf,qty,index):
    values = dataf.loc[index]
    values[3] = 1 #Here i am changing the qty to 1
    values=pd.DataFrame([values.values],columns=values.index)
    values = values.reindex(list(range(index,index+qty)),method='ffill')
    return values

In [34]:
mult_buy = buy[buy.qty > 1]
mult_sell = sell[sell.qty > 1]

In [35]:
def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]

    df = dfA.append(df_insert).append(dfB).reset_index(drop = True)

    return df

In [36]:
index = [x for x in pd.RangeIndex(13,21,1)]
index

[13, 14, 15, 16, 17, 18, 19, 20]

In [37]:
buy.drop(mult_buy.index,inplace=True)
sell.drop(mult_sell.index,inplace=True)

In [38]:
for i,qty in mult_buy['qty'].iteritems():
    print(i,qty)

In [39]:
for i,qty in mult_buy['qty'].iteritems():
    temp = make_single_1(dataf=mult_buy,qty=qty,index=i)
    update_buy = insert_row(i,buy,temp)
    buy = update_buy
    print('exit loop')
    
for i,qty in mult_sell['qty'].iteritems():
    temp = make_single_1(dataf=mult_sell,qty=qty,index=i)
    update_buy = insert_row(i,sell,temp)
    sell = update_buy
    print('exit loop')

In [40]:
#This function finally provides the corrected dataframe for each script, transaction wise
def get_trades(df,script):
    temp = df[df.script == script]
    buy = temp[temp.transaction == 'B'].reset_index()
    buy.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
    buy.order_no = buy.order_no.astype('int') 
    sell = temp[temp.transaction == 'S'].reset_index()
    sell.drop(['index','PnL','price_close','price_d'],axis=1,inplace=True)
    sell.order_no = sell.order_no.astype('int') 
    mult_buy = buy[buy.qty > 1]
    mult_sell = sell[sell.qty > 1]
    buy.drop(mult_buy.index,inplace=True)
    sell.drop(mult_sell.index,inplace=True)
    return buy, sell,mult_buy,mult_sell

In [41]:
#Next is to introduce the scripts that are carried forward for the day. 
#This function to makes the dataframe for the carried forward to this day.

def carried_forward(order_no,script,transaction,qty,price):
    loc_qty = 1
    values=pd.DataFrame({'order_no':order_no,'script':script,
                         'transaction':transaction,'qty':loc_qty,
                         'price':price},index=[0])
    values = values.reindex(list(range(qty)),method='ffill')
    return values

In [42]:
buy,sell,mult_buy,mult_sell = get_trades(df=contact17,script='SILVERMIC22APRFUT')

#calculate sell items
for i,qty in mult_sell['qty'].iteritems():
    temp = make_single_1(dataf=mult_sell,qty=qty,index=i)
    update_buy = insert_row(i,sell,temp)
    sell = update_buy
    print('exit loop')

#calculate buy items
for i,qty in mult_buy['qty'].iteritems():
    temp = make_single_1(dataf=mult_buy,qty=qty,index=i)
    update_buy = insert_row(i,buy,temp)
    buy = update_buy
    print('exit loop')
buy.sort_values('order_no',inplace=True)
sell.sort_values('order_no',inplace=True)

In [43]:
apr_forward = carried_forward(order_no='forward',script='SILVERMIC22APRFUT',transaction='S',
                              qty=6,price=67517)
sell = insert_row(0,sell,apr_forward)

In [44]:
apr_forward_close = carried_forward(order_no='close',script='SILVERMIC22APRFUT',transaction='B',
                              qty=6,price=68625)
buy = insert_row(0,buy,apr_forward_close)

In [45]:
final_comp_APR = buy.merge(right=sell,right_index=True,left_index=True,how='left')
final_comp_APR['PnL'] = final_comp_APR.price_y - final_comp_APR.price_x
final_comp_APR.to_excel('Apr_Script.xls')

In [46]:
final_comp_APR

,order_no_x,script_x,transaction_x,qty_x,price_x,order_no_y,script_y,transaction_y,qty_y,price_y,PnL
0,close,SILVERMIC22APRFUT,B,1,68625.0,forward,SILVERMIC22APRFUT,S,1,67517.0,-1108.0
1,close,SILVERMIC22APRFUT,B,1,68625.0,forward,SILVERMIC22APRFUT,S,1,67517.0,-1108.0
2,close,SILVERMIC22APRFUT,B,1,68625.0,forward,SILVERMIC22APRFUT,S,1,67517.0,-1108.0
3,close,SILVERMIC22APRFUT,B,1,68625.0,forward,SILVERMIC22APRFUT,S,1,67517.0,-1108.0
4,close,SILVERMIC22APRFUT,B,1,68625.0,forward,SILVERMIC22APRFUT,S,1,67517.0,-1108.0
5,close,SILVERMIC22APRFUT,B,1,68625.0,forward,SILVERMIC22APRFUT,S,1,67517.0,-1108.0
6,210014749,SILVERMIC22APRFUT,B,1,68600.0,210046921,SILVERMIC22APRFUT,S,1,68532.0,-68.0
7,210014750,SILVERMIC22APRFUT,B,1,68604.0,210053852,SILVERMIC22APRFUT,S,1,68674.0,70.0
8,210015963,SILVERMIC22APRFUT,B,1,68557.0,210055200,SILVERMIC22APRFUT,S,1,68703.0,146.0
9,210028459,SILVERMIC22APRFUT,B,1,68425.0,210056894,SILVERMIC22APRFUT,S,1,68785.0,360.0


In [47]:
buy, sell,mult_buy,mult_sell = get_trades(df=contact17,script='SILVERMIC22JUNFUT')

#calculate sell items
for i,qty in mult_sell['qty'].iteritems():
    temp = make_single_1(dataf=mult_sell,qty=qty,index=i)
    update_buy = insert_row(i,sell,temp)
    sell = update_buy
    print('exit loop')

#calculate buy items
for i,qty in mult_buy['qty'].iteritems():
    temp = make_single_1(dataf=mult_buy,qty=qty,index=i)
    update_buy = insert_row(i,buy,temp)
    buy = update_buy
    print('exit loop')
buy.sort_values('order_no',inplace=True)
sell.sort_values('order_no',inplace=True)

exit loop


In [48]:
jun_forward = carried_forward(order_no='forwarded',script='SILVERMIC22JUNFUT',transaction='B',
                              qty=6,price=68327)
jun_forward_close= carried_forward(order_no='close',script='SILVERMIC22JUNFUT',transaction='B',
                              qty=6,price=69410)

In [49]:
sell = insert_row(0,sell,jun_forward)
buy = insert_row(0,buy,jun_forward_close)

final_comp_Jun = buy.merge(right=sell,right_index=True,left_index=True,how='left')
final_comp_Jun['PnL'] = final_comp_Jun.price_x - final_comp_Jun.price_y
final_comp_Jun.to_excel('Jun_Script.xls')

In [50]:
final_comp_Jun

,order_no_x,script_x,transaction_x,qty_x,price_x,order_no_y,script_y,transaction_y,qty_y,price_y,PnL
0,close,SILVERMIC22JUNFUT,B,1,69410.0,forwarded,SILVERMIC22JUNFUT,B,1,68327.0,1083.0
1,close,SILVERMIC22JUNFUT,B,1,69410.0,forwarded,SILVERMIC22JUNFUT,B,1,68327.0,1083.0
2,close,SILVERMIC22JUNFUT,B,1,69410.0,forwarded,SILVERMIC22JUNFUT,B,1,68327.0,1083.0
3,close,SILVERMIC22JUNFUT,B,1,69410.0,forwarded,SILVERMIC22JUNFUT,B,1,68327.0,1083.0
4,close,SILVERMIC22JUNFUT,B,1,69410.0,forwarded,SILVERMIC22JUNFUT,B,1,68327.0,1083.0
5,close,SILVERMIC22JUNFUT,B,1,69410.0,forwarded,SILVERMIC22JUNFUT,B,1,68327.0,1083.0
6,210014685,SILVERMIC22JUNFUT,B,1,69387.0,210015627,SILVERMIC22JUNFUT,S,1,69342.0,45.0
7,210015708,SILVERMIC22JUNFUT,B,1,69346.0,210015628,SILVERMIC22JUNFUT,S,1,69341.0,5.0
8,210015729,SILVERMIC22JUNFUT,B,1,69340.0,210015629,SILVERMIC22JUNFUT,S,1,69340.0,0.0
9,210015768,SILVERMIC22JUNFUT,B,1,69340.0,210015630,SILVERMIC22JUNFUT,S,1,69340.0,0.0


Things I have learnt:

Insert method needs values in numpy array format.

In order to convert the series values in dataframe, have to encapsulate the Series inside []  and then pass into pd.dataframe()

When the index is referred for the existing dataframe, the location of the data insertion is not always correct. Need to use alternate method

The way of iterating the dataframe is using df.iteritems() method. This will give the complete row of the data frame, and df.index() will refer to index in the dataframe

Droping the row is simply df.drop(row_index,inplace=True)

index = [x for x in pd.RangeIndex(13,21,1)] using this route 

What I need to learn is, there a way to position the data exactly in the location we want.

Using the Re-index and the append method in tandem with loc indexes, the rows can be effectively inserted into the dataframe... That is a long ardous way to learn about data frame manipulations... 

In [51]:
# extract the lines that having qty more than one,using the make_single()
#get qty first
for i, qty in enumerate(buy.qty):
    if qty > 1:
        values = buy.iloc[i]
        values[3] = 1 #Here i am changing the qty to 1
        df=pd.DataFrame()
        for q in range(qty-1):
            df = pd.concat([df,values])
        buy.drop(loc_i+1,inplace=True)

In [52]:
# extract the lines that having qty more than one,using the make_single()
#get qty first
for i, qty in enumerate(buy.qty):
    if qty > 1:
        print('entry index {}'.format(i))
        print(qty)
        values = buy.iloc[i]
        values[3] = 1 #Here i am changing the qty to 1
        loc_i = i
        for q in range(qty-1):
            df = pd.DataFrame(np.insert(buy.values, loc_i, values=values, axis=0))
            df.columns = buy.columns
            buy = df #re-assign to the source dataframe
            loc_i = loc_i+1
        print('local index {}'.format(loc_i))
        buy.drop(loc_i+1,inplace=True)

In [53]:
import os
directory = '../input/contract-notes'
for filename in os.listdir(directory):
    if filename.endswith(".pdf"): 
         print(os.path.join(directory, filename))

../input/contract-notes/2022-03-08-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-09-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-07-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-17-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-16-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-15-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-11-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-10-contract-notes_DM3955.pdf
../input/contract-notes/2022-03-14-contract-notes_DM3955.pdf
